In [ ]:
# Alexis

In this notebook Alexis will join a beacon cohort and generate an offline, sidecar did:btc1 DID that includes this beacon address in the DID document. Then Alexis will participate in the aggregation protocol of the beacon cohort, authorizing an aggregated update that does not contain an update for his DID.

This notebook should be run together with the Satoshi and Aggregator notebooks, together they implement the following high level steps:

The high level steps required for aggregation are as follows:

- **A. Create a DID, already bound to an aggregator (Controller)**
- B. Aggregator creates cohort (Aggregator)
- **C. Update that DID document (Controller)**
- D. Aggregate Updates (Aggregator)
- R. Resolve Updated DID Document (Resolver)

Steps A and B are two halves of an interactive protocol in which a DID controller joins a beacon cohort and includes the resulting beacon address in an offline DID they are creating. Similarly C and D are two halves of an interactive protocol whereby a DID controller submits an update to the Aggregator, who aggregates it along with other updates from cohort participants, requests authorization and broadcasts the aggregated update through a bitcoin spend from the beacon address.

This notebook implements the DID controller halves these flows (A and C in bold). It should be run alongside the notebooks for the Aggregator and Satoshi.

**The communication protocols required to facilitate interaction between an aggregator and a DID controller still need to be defined (Our initial hope is to leverage NOSTR). For these notebooks we use the Copy + Paste protocol. You the notebook runner will be told when to go to different notebooks, execute cells and copy across payloads that can be used in other notebooks.**

This notebook is similar to the Satoshi notebook, except that Satoshi will be creating and submiting a DID update.



In [1]:
import sys
import os

# Get the path to the Notebooks directory
notebooks_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the Notebooks directory to the sys.path
sys.path.append(notebooks_path)

# Prerequisites

DID controllers are expected to have access to a Hardware Key that can be used to generate Secp256k1 keys. Ideally this would be something along the lines of a ledger.

In [2]:
from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey

In [3]:
## Run this if you want a new hardware key
mnemonic = secure_mnemonic()

mnemonic = "offer apple busy alarm lawsuit fence deny marriage beauty divorce essay message task believe buyer error planet energy frozen gap bronze tissue umbrella access"
root_hdpriv = HDPrivateKey.from_mnemonic(mnemonic, network="signet")

print("Mnemonic : ", mnemonic)

Mnemonic :  offer apple busy alarm lawsuit fence deny marriage beauty divorce essay message task believe buyer error planet energy frozen gap bronze tissue umbrella access


# A. Create an offline DID, already bound to an aggregator

This can be broken down into five steps:

1. Find an aggregator
2. Join an aggregation cohort
3. Create DID Document
4. Create actual DID
5. Register DID with aggregator
6. Use DID



## A1. Find an Aggregator

Aggregators advertise open beacons that participants can join. The participant needs to identify a suitable advertised beacon cohort and then proceed to join that cohort.

To do this there needs to be an aggregator advertising beacon cohorts. 



### A1.1. Fetch Beacon Advert

**First run through steps B1. and B2. of the [Aggregator](./Aggregator.ipynb) notebook to define the beacon advert. Use the Copy+Paste protocol to announce this advert to DID controllers.**

In [4]:
### THIS SHOULD BE COPIED FROM THE AGGREGATOR ###

beacon_advert={'id': 'SOMEBEACONID', 'type': 'SMTAggregatorBeacon', 'cohort_size': 2, 'btc_network': 'signet', 'frequency': '1 per month', 'return_address': 'SMT_Aggregator => Copy + Paste'}


In [5]:
beacon_advert

{'id': 'SOMEBEACONID',
 'type': 'SMTAggregatorBeacon',
 'cohort_size': 2,
 'btc_network': 'signet',
 'frequency': '1 per month',
 'return_address': 'SMT_Aggregator => Copy + Paste'}

## A2. Join aggregator cohort

### A2.1 Create aggregation participant key

A Secp256k1 key is required, this will be used by the aggregator to contruct and n-of-n p2tr address, where n is the number of cohort participant keys.

In [6]:
from buidl.helper import encode_base58, decode_base58, encode_base58_checksum

# We could define a specific purpose (Current purposes are for different script types)
didkey_purpose = "11"

cohort_participation_sk = root_hdpriv.get_private_key(didkey_purpose, address_num=2)
cohort_participation_pk = cohort_participation_sk.point

print("Secp256k1 PrivateKey", cohort_participation_sk.hex())
print("Secp256k1 Public Key", cohort_participation_pk.__repr__())

Secp256k1 PrivateKey 921a84cad03f30aaa88f46fd096fd9b1553c29e17881784c4702f12d2812ea23
Secp256k1 Public Key S256Point(0242f96da7b5a849b97044a9e494ac711f14fd5189e468b84fcc8fad4b007302d6)


### A2.2 Construct Beacon OptIn message to aggregator

Includes:
- Advert ID: Uniquely identifies the advert within the context of the aggregator
- Cohort Participation Public Key: Hex representation of a Secp256k1 public key
- signature: A signature from the cohort particpation public key
- return_interact?: How the aggregator can interact with participant
- did?

**Question:** Should OptIn message be a VC?

In [ ]:
hex_pubkey = cohort_participation_pk.sec().hex()

beacon_opt_in = {
    "advert_id": beacon_advert["id"],
    "participant_pubkey": hex_pubkey,
    # TODO: this could be a DID serviceEndpoint?
    "return_interact": "Alexis => Copy+Paste"
    # TODO: some authentication?
}

### A2.3 Sign Beacon OptIn

In [8]:
## TODO: sign beacon opt in
# This is an example of how we might sign
# We really need a VC or something
# All needs to be worked out as part of the protocol

import jcs

from buidl.helper import sha256

data = jcs.canonicalize(beacon_opt_in)

hash_data = sha256(data)
sig = cohort_participation_sk.sign_schnorr(hash_data)

In [9]:
sig.serialize()

b'\x8f\xcb\x8c.\x15wX\xb2_~\xfb\xdd\xd9\x14h\xd7\xa9\xb0\x07\x01dv\xe4$R\xe4\x84\x0f\x0b\x83\xf8e\xc2N\xc3H\xac>\x1fZ>\x94\x0c\xe6\xae\rq\xfc\x0fN^"\xcds\x08\xe4=\xfc\xc0{E\xda\xc4^'

### A2.4 Send Beacon OptIn payload to Aggregator

Copy+Paste the output below into step B3 in the [Aggregator](./Aggregator.ipynp) notebook.

In [11]:
print("Copy and Paste the below output into step B3 in the Aggregator notebook\n\n")
print(beacon_opt_in)

Copy and Paste the below output into step B3 in the Aggregator notebook


{'advert_id': 'SOMEBEACONID', 'participant_pubkey': '0242f96da7b5a849b97044a9e494ac711f14fd5189e468b84fcc8fad4b007302d6', 'return_interact': 'Alexis => Copy+Paste'}


### A2.5 Get Cohort Set Payload

Copy+Paste the cohort set payload output from step B5.3 of the [Aggregator](./Aggregator.ipynb) notebook

In [12]:
cohort_set_payload = {'advert_id': 'SOMEBEACONID', 'participant_pubkeys': ['0242f96da7b5a849b97044a9e494ac711f14fd5189e468b84fcc8fad4b007302d6', '02f0a96d61fd8451566b861bc78f2e2a34679b841dcd7397f2ed9af95225d8da72'], 'beacon_address': 'tb1p8fx4cmaqt0hsrnw6kprqcjtrtrhkpjvclzwra58yz8klp4r7vv4qs3hpz5'}

### A2.6 Check Advert ID

AdvertID of cohort set payload should match the ID for the beacon advert the participant opted into.

In [13]:
assert(cohort_set_payload["advert_id"] == beacon_advert["id"])

### A2.7 Verify Signature

In [14]:
# TODO: Verify Signature

### A2.8 Verify Beacon Address

In [15]:
from buidl.ecc import S256Point

cohort_hex_pks = cohort_set_payload["participant_pubkeys"]


In [16]:

# Check own PK in the set of cohort keys
assert(hex_pubkey in cohort_hex_pks)


In [17]:
cohort_pks = [S256Point.parse(bytes.fromhex(hex_key)) for hex_key in cohort_hex_pks]

from buidl.taproot import MuSigTapScript, TapRootMultiSig, P2PKTapScript

musig = MuSigTapScript(cohort_pks)

### ADDITIONAL STEP BECAUSE OF BUG IN BUIDL LIBRARY

p2tr musig2 must include a tweak

Note: per [BIP341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki) you should always include a tweak. In the case where tapscripts are not required the tweak should commit to an unspendable ScriptPath. Specifically

> If the spending conditions do not require a script path, the output key should commit to an unspendable script path instead of having no script path. This can be achieved by computing the output key point as Q = P + int(hashTapTweak(bytes(P)))G. [23]

I think the tweak construction that is being used in the below cell is not quite right

In [18]:
tr_multisig = TapRootMultiSig(cohort_pks, len(cohort_pks))
internal_pubkey = tr_multisig.default_internal_pubkey
branch = tr_multisig.musig_tree()
tr_merkle_root = branch.hash()

network = beacon_advert["btc_network"]
p2tr_beacon_address = internal_pubkey.p2tr_address(tr_merkle_root,network=network)



In [19]:
assert(cohort_set_payload["beacon_address"] == p2tr_beacon_address)

In [20]:
print("Beacon Address Set : " + p2tr_beacon_address)

Beacon Address Set : tb1p8fx4cmaqt0hsrnw6kprqcjtrtrhkpjvclzwra58yz8klp4r7vv4qs3hpz5


### OPTIONAL: Fund beacon address

Depending on the beacon T&Cs each participant might be required to fund the beacon address. Or, as in this implementation, the aggregator may fund the address themselves

## A3. Create DID Document

The cohort participant creates an external DID document with arbitrary content and includes the beacon address as a service endpoint within the document.

### A3.1 Create arbitrary external DID document content

Including:
- Verification Methods
- Verification Relationships
- Service Endpoints

In [21]:
intermediate_did_doc = {}


#### A3.1.1 Set the DID context

TODO: define the context for did:btc, need to specify the beacon service types

In [22]:
intermediate_did_doc["@context"] = [
    "https://www.w3.org/ns/did/v1",
    # "<didbtc_context>"
]

#### A3.1.1 Create verification method

TODO: potentially change the verification method into Multikey pending outcome of https://gl1.dcdpr.com/btcr/btcr/-/issues/31

In [23]:
initial_vm_privkey = root_hdpriv.get_private_key(didkey_purpose, address_num=4)
initial_vm_pubkey = initial_vm_privkey.point
initial_vm_privkey.wif(compressed=False)

verificationMethod = {}
verificationMethod["id"] = '#initialKey'
verificationMethod["type"] = "JsonWebKey"
x = initial_vm_pubkey.sec().hex()

kty = 'EC'
crv = 'secp256k1'
jwkObject = {
    "kty": kty,
    "crv": crv,
    "x": x
}

verificationMethod["publicKeyJwk"] = jwkObject

intermediate_did_doc["verificationMethod"] = [verificationMethod]

#### A3.1.2 Define Verification Relationships

In [27]:
verificationMethod

{'id': '#initialKey',
 'type': 'JsonWebKey',
 'publicKeyJwk': {'kty': 'EC',
  'crv': 'secp256k1',
  'x': '029a182c787912e27145f6d82a062cbda960f4bb2c24ef5cd9ea4a667b5a0d101e'}}

In [28]:
intermediate_did_doc["authentication"] = [verificationMethod["id"]]
intermediate_did_doc["assertionMethod"] = [verificationMethod["id"]]
intermediate_did_doc["capabilityInvocation"] = [verificationMethod["id"]]
intermediate_did_doc["capabilityDelegation"] = [verificationMethod["id"]]

#### A3.1.3 Create Service Endpoint

In [34]:
services = [];

## TOOD:

#### A3.1.4 Create Single Update Beacon Service

This is RECOMMENDED and acts as an escape hatch so that DID controllers are always able to update their DID even if the aggregated beacons they participate in fail.

In [35]:
# Create beacon keypair
single_beacon_private_key = root_hdpriv.get_private_key(didkey_purpose, address_num=3)
single_beacon_public_key = single_beacon_private_key.point



In [36]:
## Create Beacon address
# p2pkh
p2pkh_address = single_beacon_public_key.p2pkh_script().address(network=network)
# p2wpkh
p2wpkh_address = single_beacon_public_key.p2wpkh_address(network=network)
# p2tr
p2tr_address = single_beacon_public_key.p2tr_address(network=network)

## Markus: Maybe this could go in the Metadata DIDDocument

In [37]:
p2pkh_uri = 'bitcoin:' + p2pkh_address
p2wpkh_uri = 'bitcoin:' + p2wpkh_address
p2tr_uri = 'bitcoin:' + p2tr_address

p2pkh_beacon = {
    "id": "#initial_p2pkh",  # Can be anything
    "type": "SingletonBeacon",
    "serviceEndpoint": p2pkh_uri  
}
p2wpkh_beacon = {
    "id": "#initial_p2wpkh", 
    "type": "SingletonBeacon",
    "serviceEndpoint": p2wpkh_uri
}

p2tr_beacon = {
    "id": "#initial_p2tr", 
    "type": "SingletonBeacon",
    "serviceEndpoint": p2tr_uri
}

services.append(p2pkh_beacon)
services.append(p2wpkh_beacon)
services.append(p2tr_beacon)

## A3.1.5 Add Aggregated Beacon Service

Use the beacon address recieved from step A2.

In [38]:
p2tr_aggregated_beacon_uri = 'bitcoin:' + p2tr_beacon_address
beacon_type = beacon_advert["type"]
aggregated_beacon = {
    "id": "#smt_aggregated",
    "type": beacon_type,
    "serviceEndpoint": p2tr_aggregated_beacon_uri
}

services.append(aggregated_beacon)

In [39]:
services

[{'id': '#initial_p2pkh',
  'type': 'SingletonBeacon',
  'serviceEndpoint': 'bitcoin:mqzVvotMQAWcxPTHUdBs1Nq7Mn7K3iZeCG'},
 {'id': '#initial_p2wpkh',
  'type': 'SingletonBeacon',
  'serviceEndpoint': 'bitcoin:tb1qwtnw2v33j49jsava5ypqtcug3ndm5akavvtp43'},
 {'id': '#initial_p2tr',
  'type': 'SingletonBeacon',
  'serviceEndpoint': 'bitcoin:tb1pyvh0l5s4m7w293muq2u29dzfyfhhp46y4jm9lt47876h70qndm3qztafja'},
 {'id': '#smt_aggregated',
  'type': 'SMTAggregatorBeacon',
  'serviceEndpoint': 'bitcoin:tb1p8fx4cmaqt0hsrnw6kprqcjtrtrhkpjvclzwra58yz8klp4r7vv4qs3hpz5'}]

In [40]:
aggregated_beacon

{'id': '#smt_aggregated',
 'type': 'SMTAggregatorBeacon',
 'serviceEndpoint': 'bitcoin:tb1p8fx4cmaqt0hsrnw6kprqcjtrtrhkpjvclzwra58yz8klp4r7vv4qs3hpz5'}

In [41]:
intermediate_did_doc["service"] = services

In [43]:
print("Intermediate DID document\n")
print(intermediate_did_doc)

Intermediate DID document

{'@context': ['https://www.w3.org/ns/did/v1'], 'verificationMethod': [{'id': '#initialKey', 'type': 'JsonWebKey', 'publicKeyJwk': {'kty': 'EC', 'crv': 'secp256k1', 'x': '029a182c787912e27145f6d82a062cbda960f4bb2c24ef5cd9ea4a667b5a0d101e'}}], 'authentication': ['#initialKey'], 'assertionMethod': ['#initialKey'], 'capabilityInvocation': ['#initialKey'], 'capabilityDelegation': ['#initialKey'], 'service': [{'id': '#initial_p2pkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:mqzVvotMQAWcxPTHUdBs1Nq7Mn7K3iZeCG'}, {'id': '#initial_p2wpkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:tb1qwtnw2v33j49jsava5ypqtcug3ndm5akavvtp43'}, {'id': '#initial_p2tr', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:tb1pyvh0l5s4m7w293muq2u29dzfyfhhp46y4jm9lt47876h70qndm3qztafja'}, {'id': '#smt_aggregated', 'type': 'SMTAggregatorBeacon', 'serviceEndpoint': 'bitcoin:tb1p8fx4cmaqt0hsrnw6kprqcjtrtrhkpjvclzwra58yz8klp4r7vv4qs3hpz5'}]}


## A4. Create DID

### A4.1 Hash the ID-less DID Document

TODO: Instead of a sha256 hash of the intermediate DID document, we may use the IPFS CID generation

In [44]:
from buidl.helper import sha256, str_to_bytes, bytes_to_str, encode_base58
import json
from libbtc1.bech32 import encode_bech32_identifier, decode_bech32_identifier


# TODO: canonicalization?
data = str_to_bytes(json.dumps(intermediate_did_doc))


hashed_initial_doc = sha256(data)


## TODO: Move to bech32
method_identifier = encode_bech32_identifier("external", hashed_initial_doc)

In [45]:
method_identifier

'x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u'

## IPFS Option

In [52]:
from ipfs_cid import cid_sha256_hash

data = str_to_bytes(json.dumps(intermediate_did_doc))

cid = cid_sha256_hash(data)
cid_bytes = str_to_bytes(cid)

ipfs_method_identifier = encode_bech32_identifier("external", cid_bytes)

print("IPFS Method Identifier : ", ipfs_method_identifier)

IPFS Method Identifier :  x1vfskv6mjv45kvdnsvdcxuutcvg6k2urvvdcxzaphx4kxs6t9v4skgarwvvehz7fkwpcxy6mgdenny6ekv3uxs6pkwa6hsmgqaa9sy


In [50]:
cid

'bafkreif6pcpnqxb5eplcpat75lhieeadtnc3qy6ppbkhng2k6dxhh6wuxm'

## Note: It is possible to go from a hash to an IPFS CID

Therefore I do not believe that the bech32 encoding of the method identifier should encode the IPFS CID. It can just be the SHA256 hash, that some resolvers may translate into an IPFS CID to query the network. Although, this does bring some challenges as there are many valid IPFS CIDs for a single hash. Hmmm, perhaps it should be an IPFS CID then


In [35]:
from ipfs_cid import cid_sha256_wrap_digest

cid_wrap_digest = cid_sha256_wrap_digest(hashed_initial_doc)

assert(cid == cid_wrap_digest)


### A4.3 Set DID as id in the DID document

Do we set the controller property on the verificationMethod aswell? https://gl1.dcdpr.com/btcr/btcr/-/issues/167

In [46]:
did_btc1 = f"did:btc1:{method_identifier}"
did_btc1

'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u'

In [47]:
import copy

did_document = copy.deepcopy(intermediate_did_doc)
did_document["id"] = did_btc1

# TODO: What about the verificationMethod controller properties? Should this also be set as the DID?
did_document["verificationMethod"][0]["controller"] = did_btc1



In [48]:
print("Sidecar DID document for " + did_btc1 + "\n\n")
print(did_document)

Sidecar DID document for did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u


{'@context': ['https://www.w3.org/ns/did/v1'], 'verificationMethod': [{'id': '#initialKey', 'type': 'JsonWebKey', 'publicKeyJwk': {'kty': 'EC', 'crv': 'secp256k1', 'x': '029a182c787912e27145f6d82a062cbda960f4bb2c24ef5cd9ea4a667b5a0d101e'}, 'controller': 'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u'}], 'authentication': ['#initialKey'], 'assertionMethod': ['#initialKey'], 'capabilityInvocation': ['#initialKey'], 'capabilityDelegation': ['#initialKey'], 'service': [{'id': '#initial_p2pkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:mqzVvotMQAWcxPTHUdBs1Nq7Mn7K3iZeCG'}, {'id': '#initial_p2wpkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:tb1qwtnw2v33j49jsava5ypqtcug3ndm5akavvtp43'}, {'id': '#initial_p2tr', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:tb1pyvh0l5s4m7w293muq2u29dzfyfhhp46y4jm9lt47876h70qndm3qztafja'}, {'id': '#smt_aggreg

### A4.4 Store DID and initial DID document



In [49]:
## Notebook acts as the store in this example
serialized_history = str_to_bytes(json.dumps(intermediate_did_doc)).hex()

f = open(did_btc1, "a")
f.write(serialized_history)
f.close()


## A5. Register DID with Aggregator

Aggregator needs to know all DIDs that a cohort participant intends to use the aggregated beacon with to aggregate updates. These are the DIDs that the aggregator will generate and provide proofs of (non)inclusion for.

### A5.1 Construct DID registration payload

This includes:

- beacon_address: Identifies the beacon
- participant_pk: Identifies the cohort participant in the beacon
- did: The DID string that the cohort participant wishes to register

TODO: Anything else?

In [49]:
did_registration_payload = {
    "beacon_address": p2tr_beacon_address,
    "participant_pk": hex_pubkey,
    "did": did_btc1
}

### A5.2 Sign the DID registration payload

This is a TODO.

**Question:** Sign with what? 
- Participant_pk
- did_btc

I think either participant_pk, or both.

In [50]:
# TODO: Sign the payload

### A5.3 Send DID registration payload to the aggregator

Use the Copy+Paste protocol to communicate the payload output below to section B6. of the [Aggregator](./SMT_Aggregator.ipynb) notebook.

In [52]:
print("Use the Copy+Paste protocol to communicate the payload output below to section B6. of the Aggregator notebook.\n\n")
print(did_registration_payload)

Use the Copy+Paste protocol to communicate the payload output below to section B6. of the Aggregator notebook.


{'beacon_address': 'tb1p8fx4cmaqt0hsrnw6kprqcjtrtrhkpjvclzwra58yz8klp4r7vv4qs3hpz5', 'participant_pk': '0242f96da7b5a849b97044a9e494ac711f14fd5189e468b84fcc8fad4b007302d6', 'did': 'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u'}


## A6 Use DID

- Include DID document with DID
  - As the DID Document is updated, the entire DID Document update history must be presented along with the DID. This is sidecar.
- Verifier can verify the provided DID Document matches the hash


## COMPLETED: A. Create a DID, already bound to an aggregator

The did:btc1 created in step A is an offline, sidecar and updatable DID document. The DID controller can use an aggregator to publish updates for reduced costs and increased scalability.

# C. Update DID document

To update the DID document created in step A requires the following steps:

1. Create updated DID document
2. Create update payload
3. Create DID update invocation
4. Publish update
5. Store update
6. Watch BTC to verify publication

To update a DID document using the SMTAggregatedBeacon requires interaction with the Aggregator following the steps from D Aggregate Updates. Again, we will use the Copy+Paste protocol to simulate this interaction.

## C.1 Create updated DID document

- Include any additions to the DID document
- Any deletions from the DID document
- Can add/remove beacons from the service array

In [53]:
import copy
updated_diddocument = copy.deepcopy(did_document)

# Adding a new service
linked_domain = {
    "id":"#linked-domain",
    "type": "LinkedDomains", 
    "serviceEndpoint": "https://contact-me.com"    
}

updated_diddocument["service"].append(linked_domain)

In [54]:
updated_diddocument

{'@context': ['https://www.w3.org/ns/did/v1'],
 'verificationMethod': [{'id': '#initialKey',
   'type': 'JsonWebKey',
   'publicKeyJwk': {'kty': 'EC',
    'crv': 'secp256k1',
    'x': '029a182c787912e27145f6d82a062cbda960f4bb2c24ef5cd9ea4a667b5a0d101e'},
   'controller': 'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u'}],
 'authentication': ['#initialKey'],
 'assertionMethod': ['#initialKey'],
 'capabilityInvocation': ['#initialKey'],
 'capabilityDelegation': ['#initialKey'],
 'service': [{'id': '#initial_p2pkh',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:mqzVvotMQAWcxPTHUdBs1Nq7Mn7K3iZeCG'},
  {'id': '#initial_p2wpkh',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:tb1qwtnw2v33j49jsava5ypqtcug3ndm5akavvtp43'},
  {'id': '#initial_p2tr',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:tb1pyvh0l5s4m7w293muq2u29dzfyfhhp46y4jm9lt47876h70qndm3qztafja'},
  {'id': '#smt_aggregated',
   'type': 'SMTAggregatorBeacon',
   'servi

## C2 Create DID update payload

- JSON Patch transformation from current document to new one 
- Bump version number

 
**Question** would this be better to version in the payload?
DID Document propagates to caller of resolver. That could be useful, especially for enabling resolution by version #.
https://www.w3.org/TR/did-core/#dfn-versionid 


In [55]:
# This is saying add linkedDomanSE to the end of the service list of a json document.

service_path = f"/service/{len(did_document["service"])}"

update_patch = [{'op': 'add', 'path': service_path, 'value': linked_domain}]


print(update_patch)

[{'op': 'add', 'path': '/service/4', 'value': {'id': '#linked-domain', 'type': 'LinkedDomains', 'serviceEndpoint': 'https://contact-me.com'}}]


In [101]:
# This is the first update
# If you want to do another update you MUST bump the version
version = 1

sourceHash = sha256(str_to_bytes(json.dumps(did_document))).hex()

In [102]:
sourceHash

'a4782e8b3a6061e33aac0da0f2260c05967402b21da4c911d3fbb23b8b448aef'

In [103]:
did_update_payload = {
    '@context': [
        'https://w3id.org/security/v2',
        'https://w3id.org/zcap/v1',
        'https://w3id.org/json-ld-patch/v1'
        # TODO did:btc1 zcap context
    ],
    'patch': update_patch,
    # TODO: this might not go here?
    'sourceHash': sourceHash,
    'versionId': version
};

## C3. Create DID Update Invocation

### C3.1 Deterministically Generate Root Capability for DID document

This is a root capability for a specific did:btc1 to invoke a capability to update that did:btc's DID document

In [59]:
import urllib
root_capability = {
  "@context": "https://w3id.org/security/v2",
  "id": f"urn:zcap:root:{urllib.parse.quote(did_btc1)}",
  "controller": did_btc1,
  "invocationTarget": did_btc1
};
root_capability

{'@context': 'https://w3id.org/security/v2',
 'id': 'urn:zcap:root:did%3Abtc1%3Ax1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u',
 'controller': 'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u',
 'invocationTarget': 'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u'}

### C3.2 Invoke root capability over the DID update payload

**Note:** There are no libraries for this in python. I have a separate POC that demonstrates how to achieve this using libraries for Digital Bazaar in JavaScript.

In [60]:
did_update_invocation = copy.deepcopy(did_update_payload)


## This would be generated through some signing library that uses the verification method
did_update_invocation["proof"] = {
    'type': 'SchnorrSecp256k12024',
    'created': '2024-01-04T17:59:51Z',
    'verificationMethod': did_document["verificationMethod"][0]["id"],
    'proofPurpose': 'capabilityInvocation',
    'capability': root_capability["id"],
    'invocationTarget': did_btc1,
    'capabilityAction': 'write',
    'proofValue': 'z381yXYmxU8NudZ4HXY56DfMN6zfD8syvWcRXzT9xD9uYoQToo8QsXD7ahM3gXTzuay5WJbqTswt2BKaGWYn2hHhVFKJLXaDz'
  }



In [61]:
did_update_invocation

{'@context': ['https://w3id.org/security/v2',
  'https://w3id.org/zcap/v1',
  'https://w3id.org/json-ld-patch/v1'],
 'patch': [{'op': 'add',
   'path': '/service/4',
   'value': {'id': '#linked-domain',
    'type': 'LinkedDomains',
    'serviceEndpoint': 'https://contact-me.com'}}],
 'version': 1,
 'proof': {'type': 'SchnorrSecp256k12024',
  'created': '2024-01-04T17:59:51Z',
  'verificationMethod': '#initialKey',
  'proofPurpose': 'capabilityInvocation',
  'capability': 'urn:zcap:root:did%3Abtc1%3Ax1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u',
  'invocationTarget': 'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u',
  'capabilityAction': 'write',
  'proofValue': 'z381yXYmxU8NudZ4HXY56DfMN6zfD8syvWcRXzT9xD9uYoQToo8QsXD7ahM3gXTzuay5WJbqTswt2BKaGWYn2hHhVFKJLXaDz'}}

## C4. Publish DID update

Publishing a DID update using an aggregated beacon requires coordination of a multiparty cryptographic protocol to produce a MuSig2 signature that is a valid spend from the n-of-n beacon address.

From the cohort participants perspective this can can be broken down into the following substeps:

1. Select beacon from current DID document with suitable cohort policy
2. Send invocation to beacon aggregator
3. Receive authorize beacon signal request
4. Verify contents of pending beacon signal
5. Generate MuSig2 aggregated nonce
6. Generate MuSig2 signing data
7. Authorize pending beacon signal
8. Return partially signed bitcoin transaction

BIP0327 defines the MuSig2 signing protocol in more detail - https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki

### C4.1 Select beacon from the current DID document with a suitable cohort policy

TODO: need to identify the aggregator communication endpoint from the identified beacon service. For this example we just use the Copy+Paste protocol to interact with the [Aggregator](./Aggregator.ipynb)

In [62]:
beacon_service_id = "#smt_aggregated"
beacon_service = None
for service in did_document["service"]:
    if service["id"] == beacon_service_id:
        beacon_service = service

print("Beacon service to publish to : ",beacon_service)

Beacon service to publish to :  {'id': '#smt_aggregated', 'type': 'SMTAggregatorBeacon', 'serviceEndpoint': 'bitcoin:tb1p8fx4cmaqt0hsrnw6kprqcjtrtrhkpjvclzwra58yz8klp4r7vv4qs3hpz5'}


###  C4.2 Send DID update invocation to the beacon aggregator

#### C4.2.1 Construct did update announcement

Contains:

- beacon_address: Address of the beacon
- invocation: Signed DID update invocation
- DID: The DID string being updated
- participant_pk: Uniquely identifies the cohort participant
- signature: Signature over the payload from the participant pk

In [63]:
did_update_announcement = {
    "beacon_address": p2tr_beacon_address,
    "invocation": did_update_invocation,
    "did": did_btc1,
    "participant_pk": hex_pubkey
}

#### C4.2.2 Sign the DID update payload

This is a TODO

In [64]:
## TODO: sign the DID update payload



#### C4.2.3 Send DID update payload to beacon aggregator

Copy+Paste the output below into step D1 of the [Aggregator](./Aggregator.ipynb) notebook

In [65]:
# TODO: Sign the DID update invocation using the cohort participant key


In [66]:
print("Copy+Paste the output below into step D1 of the Aggregator notebook\n\n")
print(did_update_announcement)

Copy+Paste the output below into step D1 of the Aggregator notebook


{'beacon_address': 'tb1p8fx4cmaqt0hsrnw6kprqcjtrtrhkpjvclzwra58yz8klp4r7vv4qs3hpz5', 'invocation': {'@context': ['https://w3id.org/security/v2', 'https://w3id.org/zcap/v1', 'https://w3id.org/json-ld-patch/v1'], 'patch': [{'op': 'add', 'path': '/service/4', 'value': {'id': '#linked-domain', 'type': 'LinkedDomains', 'serviceEndpoint': 'https://contact-me.com'}}], 'version': 1, 'proof': {'type': 'SchnorrSecp256k12024', 'created': '2024-01-04T17:59:51Z', 'verificationMethod': '#initialKey', 'proofPurpose': 'capabilityInvocation', 'capability': 'urn:zcap:root:did%3Abtc1%3Ax1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u', 'invocationTarget': 'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u', 'capabilityAction': 'write', 'proofValue': 'z381yXYmxU8NudZ4HXY56DfMN6zfD8syvWcRXzT9xD9uYoQToo8QsXD7ahM3gXTzuay5WJbqTswt2BKaGWYn2hHhVFKJLXaDz'}}, 'did': 'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau

### C4.3 Receive authorize beacon signal request

Paste in the output from Step D4 in the [Aggregator](./Aggregator.ipynb) notebook.

In [67]:
authorize_beacon_signal = {'session_id': 'SOME SESSION ID', 'beacon_address': 'tb1p8fx4cmaqt0hsrnw6kprqcjtrtrhkpjvclzwra58yz8klp4r7vv4qs3hpz5', 'pending_beacon_signal': '010000000001017e077ab786657bcb471d7b61e36d2450a8253d6a435288a1c5c91fe442f3cb6f0100000000ffffffff020000000000000000226a201daf0196dcfc1e7c029d1113444dc0502ca288b491d2b766cbd17b22bbacdc3942850100000000002251203a4d5c6fa05bef01cddab0460c496358ef60c998f89c3ed0e411edf0d47e632a0000000000', 'proofs': {'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u': b'\x80\x04\x95>\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\tsidenodes\x94]\x94\x8c\x17non_membership_leafdata\x94N\x8c\x0csibling_data\x94Nu.'}}

In [68]:
from buidl.tx import Tx
session_id = authorize_beacon_signal["session_id"]
pending_beacon_signal_hex = authorize_beacon_signal["pending_beacon_signal"]
pending_beacon_signal = Tx.parse_hex(pending_beacon_signal_hex)


In [69]:
# Check request from a beacon that you are participating in
assert(p2tr_beacon_address == authorize_beacon_signal["beacon_address"])

In [70]:
# TODO: verify signature from beacon aggregator on payload

## Needs to have a signature first

### C4.4 Verify authorize beacon signal request

#### C4.4.1 Check pending beacon signal is a btc transaction with a single input that is a spend from the beacon address

In [71]:
# Check has a single tx input
assert(len(pending_beacon_signal.tx_ins) == 1)

In [72]:
tx_in = pending_beacon_signal.tx_ins[0]
prev_tx = tx_in.fetch_tx(network=network)
prev_tx_out = prev_tx.tx_outs[tx_in.prev_index]
address_being_spent_from = prev_tx_out.script_pubkey.address(network="signet")

# Check tx input is a spend from the beacon address
assert(address_being_spent_from == cohort_set_payload["beacon_address"])

In [73]:
from smt.tree import SparseMerkleTree
from smt.proof import verify_proof, SparseMerkleProof
from smt.utils import DEFAULTVALUE

In [74]:
# Retrieve SMT root from bitcoin transaction

tx_out = pending_beacon_signal.tx_outs[0]
script = tx_out.script_pubkey

assert(script.commands[0] == 0x6a)
smt_root = script.commands[1]
print("Sparse Merkle Tree root : " + smt_root.hex())



Sparse Merkle Tree root : 1daf0196dcfc1e7c029d1113444dc0502ca288b491d2b766cbd17b22bbacdc39


In [75]:
import pickle
## Loop through provided proofs
proofs = authorize_beacon_signal["proofs"]

for (did, proof) in proofs.items():
    ## check key is one of the DIDs registered with this beacon
    ## TODO: handle multiple DIDs registered
    assert(did == did_btc1)
    proof_data = pickle.loads(proof)
    smt_proof = SparseMerkleProof(sidenodes=proof_data['sidenodes'], non_membership_leafdata=proof_data["non_membership_leafdata"], siblingdata=proof_data["sibling_data"])
    
    # Key to SMT leaf
    did_bytes = str_to_bytes(did)
    # Data at leaf
    did_update_invocation_bytes = str_to_bytes(json.dumps(did_update_invocation))
    verified = verify_proof(smt_proof, smt_root, did_bytes, did_update_invocation_bytes)
    print(f"Proof of inclusion for {did_btc1} Verified : {verified}")
    if not verified:
        raise Exception("Invalid SMT in beacon signal, unable to verify expected update")

Proof of inclusion for did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u Verified : True


## C4.5 Generate MuSig2 Aggregated Nonce

### C4.5.1 Generate nonce contribution

- Nonce secrets: Required for signing, should NEVER be shared
- Nonce points: Public nonce contribution. Should only be used for one signing session

In [76]:
## TOOD: Waiting for buidl library updates. Nonce generation should take in data from signing process as source of randomness
nonce_secrets, nonce_points = musig.generate_nonces()

### C4.5.2 Send nonce contribution to the aggregator

Copy+Paste the below output into Step D5.1 of the [Aggregator](.Aggregator.ipynb)

Payload includes:

- Beacon address: the address of the beacon
- Session Id: Unique identifier for this signing session provided by the aggregator (C4.5)
- Nonce points: An array of hex represented Secp256k1 points generated using the MuSig2 nonce_gen algorithm

In [77]:
pubnonces  = [point.sec().hex() for point in nonce_points]

nonce_contribution_payload = {
    "beacon_address": p2tr_beacon_address,
    "session_id": session_id,
    "nonce_points": pubnonces
}

print("Copy+Paste the below output into Step D5.1 of the Aggregator notebook \n\n")
print(nonce_contribution_payload)


Copy+Paste the below output into Step D5.1 of the Aggregator notebook 


{'beacon_address': 'tb1p8fx4cmaqt0hsrnw6kprqcjtrtrhkpjvclzwra58yz8klp4r7vv4qs3hpz5', 'session_id': 'SOME SESSION ID', 'nonce_points': ['03780f9572d340172d3afa6c8cbec192e64e8bb82db361177fccd290705325aa54', '0352f1298ecacb91b8bd50a76ca4981b3a4a60727534a7f920ae9cb63447698318']}


### C4.5.3 Receive aggregated nonce from the aggregator

Paste the output from Step D5.4 of the [Aggregator](./Aggregator.ipynb) notebook

In [78]:
aggregated_nonce_payload = {'beacon_address': 'tb1p8fx4cmaqt0hsrnw6kprqcjtrtrhkpjvclzwra58yz8klp4r7vv4qs3hpz5', 'session_id': 'SOME SESSION ID', 'agg_nonce': ['03c1f241d7703302c032bee87d48dfe458ed8bff58058153cbf8c03324279b69db', '0348c1e07d313dba2a309629b46b7ed03ca5f81b04faf2b6daa69533461684496b']}






In [79]:
assert(p2tr_beacon_address == aggregated_nonce_payload["beacon_address"])
assert(session_id == aggregated_nonce_payload["session_id"])

In [80]:
agg_nonce_hex = aggregated_nonce_payload["agg_nonce"]
agg_musig_nonce = [S256Point.parse(bytes.fromhex(hex_key)) for hex_key in agg_nonce_hex]

### C4.6 Generate MuSig2 Signing Data

See bip0327 - https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki

#### C4.6.1 Generate the bitcoin transaction sig hash

This is the data to be signed

In [81]:
from buidl.tx import Tx, SIGHASH_DEFAULT
input_index = 0
sig_hash = pending_beacon_signal.sig_hash(input_index, SIGHASH_DEFAULT)

#### C4.6.2 Generate r from aggregated nonce and Tx sig hash

In [82]:
r = musig.compute_r(agg_musig_nonce, sig_hash)


#### C4.6.3 Generate k from secret nonce, aggregated nonce and Tx sig hash


In [83]:
k = musig.compute_k(nonce_secrets, agg_musig_nonce, sig_hash)

### C4.7 Authorize pending beacon signal

Sign the btc transaction

In [84]:
partial_sig = musig.sign(cohort_participation_sk, k, r, sig_hash, tr_merkle_root)

### C4.8 Return signal authorization payload to aggregator

#### C4.8.1 Construct signal authroization payload

In [85]:
signal_authorization_payload = {
    "beacon_address": p2tr_beacon_address,
    "session_id": session_id,
    "sig": partial_sig
}

#### C4.8.2 Send payload to aggregator

Copy+Paste the below output into step D6.1 of the Aggregator notebook.

In [87]:
print("Copy+Paste the below output into step D6.1 of the Aggregator notebook.\n\n")
print(signal_authorization_payload)

Copy+Paste the below output into step D6.1 of the Aggregator notebook.


{'beacon_address': 'tb1p8fx4cmaqt0hsrnw6kprqcjtrtrhkpjvclzwra58yz8klp4r7vv4qs3hpz5', 'session_id': 'SOME SESSION ID', 'sig': 65612116011741315133739254969203259343054784735065161076729301497997339492402}


## C5. Store Update Data

The DID controller is required to store the following data for each beacon signal emitted by SMT aggregated beacons they have listed in their DID document:

- BTC TxID: The transaction ID of the beacon signal which contains the root of a sparse merkle tree in its txoutput
- Merkle proof path: Proves (non)inclusion against the SMT root in the beacon signal
- Update invocation: Either a signed update invocation or null depending if the DID controller included an update in this beacon signal.

DID controllers with many DIDs, each with potentially many different aggregation beacons are likely going to want to develop a data structure to manage this. Below is a rough example.

In [88]:
beacon_signal_txid = pending_beacon_signal.id()
beacon_signal_txid

'f77f233c26f635da4d6d106c81da46dcd460b6320c75b06c861363c04f107491'

In [89]:
proofs

{'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u': b'\x80\x04\x95>\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\tsidenodes\x94]\x94\x8c\x17non_membership_leafdata\x94N\x8c\x0csibling_data\x94Nu.'}

In [90]:
did_update_invocation

{'@context': ['https://w3id.org/security/v2',
  'https://w3id.org/zcap/v1',
  'https://w3id.org/json-ld-patch/v1'],
 'patch': [{'op': 'add',
   'path': '/service/4',
   'value': {'id': '#linked-domain',
    'type': 'LinkedDomains',
    'serviceEndpoint': 'https://contact-me.com'}}],
 'version': 1,
 'proof': {'type': 'SchnorrSecp256k12024',
  'created': '2024-01-04T17:59:51Z',
  'verificationMethod': '#initialKey',
  'proofPurpose': 'capabilityInvocation',
  'capability': 'urn:zcap:root:did%3Abtc1%3Ax1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u',
  'invocationTarget': 'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u',
  'capabilityAction': 'write',
  'proofValue': 'z381yXYmxU8NudZ4HXY56DfMN6zfD8syvWcRXzT9xD9uYoQToo8QsXD7ahM3gXTzuay5WJbqTswt2BKaGWYn2hHhVFKJLXaDz'}}

### Data structure is:

signal_txid -> {proof, update}

Signal txid's act as keys to an object with a proof and an update associated with that beacon signal.


### Load history from file if present

In [91]:
sidecar_history_filename = f"{did_btc1}_sidecar_history"
did_sidecar_history = {
    "initial_document": did_document,
    "signals": {}
}


#open and read the file after the appending:
try: 
    f = open(sidecar_history_filename, "r")
    did_sidecar_history = json.loads(f.read())
    print(f"Sidecar history found for {did_btc1}\n\n {did_sidecar_history}")
except:
    print("No sidecar history to load for DID : " + did_btc1)

No sidecar history to load for DID : did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u


In [92]:
signals = did_sidecar_history["signals"]

signals[beacon_signal_txid] = {
    "update": did_update_invocation,
    "proof": proofs[did_btc1].hex()
}

did_sidecar_history

{'initial_document': {'@context': ['https://www.w3.org/ns/did/v1'],
  'verificationMethod': [{'id': '#initialKey',
    'type': 'JsonWebKey',
    'publicKeyJwk': {'kty': 'EC',
     'crv': 'secp256k1',
     'x': '029a182c787912e27145f6d82a062cbda960f4bb2c24ef5cd9ea4a667b5a0d101e'},
    'controller': 'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u'}],
  'authentication': ['#initialKey'],
  'assertionMethod': ['#initialKey'],
  'capabilityInvocation': ['#initialKey'],
  'capabilityDelegation': ['#initialKey'],
  'service': [{'id': '#initial_p2pkh',
    'type': 'SingletonBeacon',
    'serviceEndpoint': 'bitcoin:mqzVvotMQAWcxPTHUdBs1Nq7Mn7K3iZeCG'},
   {'id': '#initial_p2wpkh',
    'type': 'SingletonBeacon',
    'serviceEndpoint': 'bitcoin:tb1qwtnw2v33j49jsava5ypqtcug3ndm5akavvtp43'},
   {'id': '#initial_p2tr',
    'type': 'SingletonBeacon',
    'serviceEndpoint': 'bitcoin:tb1pyvh0l5s4m7w293muq2u29dzfyfhhp46y4jm9lt47876h70qndm3qztafja'},
   {'id': '#smt_aggregated',
  

In [94]:
print(f"Sidecar Data for {did_btc1}") 
print(f"\nInitial DID document \n\n")
print(did_sidecar_history["initial_document"])
for signal_txid, sidecar_data in did_sidecar_history["signals"].items():
    print(f"\nSidecar Data for beacon signal with btc txid : {signal_txid}")
    print("\nDID update in signal\n")
    print(sidecar_data["update"])
    print("\nMerkle proof of update\n")
    print(sidecar_data["proof"])

Sidecar Data for did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u

Initial DID document 


{'@context': ['https://www.w3.org/ns/did/v1'], 'verificationMethod': [{'id': '#initialKey', 'type': 'JsonWebKey', 'publicKeyJwk': {'kty': 'EC', 'crv': 'secp256k1', 'x': '029a182c787912e27145f6d82a062cbda960f4bb2c24ef5cd9ea4a667b5a0d101e'}, 'controller': 'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u'}], 'authentication': ['#initialKey'], 'assertionMethod': ['#initialKey'], 'capabilityInvocation': ['#initialKey'], 'capabilityDelegation': ['#initialKey'], 'service': [{'id': '#initial_p2pkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:mqzVvotMQAWcxPTHUdBs1Nq7Mn7K3iZeCG'}, {'id': '#initial_p2wpkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:tb1qwtnw2v33j49jsava5ypqtcug3ndm5akavvtp43'}, {'id': '#initial_p2tr', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:tb1pyvh0l5s4m7w293muq2u29dzfyfhhp46y4jm9lt47876h70qndm3qztafja'}, {'id

### Save history to file for persistence across notebook sessions

In [95]:
serialized_history = json.dumps(did_sidecar_history)

f = open(sidecar_history_filename, "a")
f.write(serialized_history)
f.close()


## C6. Watch the bitcoin network for the beacon signal to be broadcast

**Most DID controllers would likley do this through their own nodes**

For these notebooks we can visit mempool.space at the url below:

In [96]:
print(f"Mempool.Space URL: https://mempool.space/{network}/tx/{beacon_signal_txid}")

Mempool.Space URL: https://mempool.space/signet/tx/f77f233c26f635da4d6d106c81da46dcd460b6320c75b06c861363c04f107491


## Step C Update DID document Completed!

You may now open the [Resolver](./Resolver.ipynb) notebook and attempt to resolve the DID created and updated in this notebook

Copy+Paste the below sidecar history required for resolution into step R1. of the Resolver notebook.

In [97]:

print(f"did_to_resolve = '{did_btc1}'")
print(f"sidecar_history = {did_sidecar_history}")

did_to_resolve = 'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u'
sidecar_history = {'initial_document': {'@context': ['https://www.w3.org/ns/did/v1'], 'verificationMethod': [{'id': '#initialKey', 'type': 'JsonWebKey', 'publicKeyJwk': {'kty': 'EC', 'crv': 'secp256k1', 'x': '029a182c787912e27145f6d82a062cbda960f4bb2c24ef5cd9ea4a667b5a0d101e'}, 'controller': 'did:btc1:x1heufakzu853avfuz0l4vaqssqwd5twrreau9ga5mftcwuul66jasl3ct6u'}], 'authentication': ['#initialKey'], 'assertionMethod': ['#initialKey'], 'capabilityInvocation': ['#initialKey'], 'capabilityDelegation': ['#initialKey'], 'service': [{'id': '#initial_p2pkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:mqzVvotMQAWcxPTHUdBs1Nq7Mn7K3iZeCG'}, {'id': '#initial_p2wpkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:tb1qwtnw2v33j49jsava5ypqtcug3ndm5akavvtp43'}, {'id': '#initial_p2tr', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:tb1pyvh0l5s4m7w293muq2u29dzfyfhhp46y4jm9lt47876h70qndm